# 08 - Introducción a Agentes de IA

## Curso de LLMs y Aplicaciones de IA

**Duración estimada:** 2-2.5 horas

---

## Índice

1. [¿Qué son los Agentes?](#intro)
2. [De RAG a Agentes](#rag2agents)
3. [Componentes de un Agente](#componentes)
4. [Planificación: Chain of Thought y ReAct](#planificacion)
5. [Primer Agente simple](#primer)
6. [Ejercicios prácticos](#ejercicios)

---

## Objetivos de aprendizaje

Al finalizar este notebook, serás capaz de:
- Entender qué diferencia a un agente de un LLM simple
- Conocer los componentes clave: planificación, memoria, herramientas
- Implementar técnicas de razonamiento como ReAct
- Crear un agente básico que use herramientas

<a name="intro"></a>
## 1. ¿Qué son los Agentes?

Un **agente de IA** es un sistema que puede:
1. **Percibir** su entorno (recibir inputs)
2. **Razonar** sobre qué hacer (planificación)
3. **Actuar** para lograr objetivos (ejecutar herramientas)
4. **Aprender** de la experiencia (memoria)

### LLM vs Agente

| LLM Simple | Agente |
|------------|--------|
| Responde preguntas | Ejecuta tareas |
| Sin estado | Con memoria |
| Solo texto | Usa herramientas |
| Un paso | Múltiples pasos |
| Pasivo | Proactivo |

<a name="rag2agents"></a>
## 2. De RAG a Agentes

RAG fue un primer paso para dar a los LLMs acceso a información externa. Los agentes expanden esta idea:

```
┌─────────────────────────────────────────────┐
│                    AGENTE                    │
├─────────────────────────────────────────────┤
│  ┌─────────────────┐                        │
│  │   PLANIFICACIÓN │ ← Descompone tareas    │
│  │   (Razonamiento)│   ReAct, CoT           │
│  └────────┬────────┘                        │
│           │                                 │
│  ┌────────▼────────┐                        │
│  │     MEMORIA     │ ← Corto y largo plazo  │
│  │  (Contexto)     │   Chat history, RAG    │
│  └────────┬────────┘                        │
│           │                                 │
│  ┌────────▼────────┐                        │
│  │   HERRAMIENTAS  │ ← Búsqueda, cálculo,   │
│  │    (Acciones)   │   APIs, código         │
│  └─────────────────┘                        │
└─────────────────────────────────────────────┘
```

In [ ]:
# Install required libraries
!pip install -q langchain langchain-groq langchain-community

In [ ]:
import os
from getpass import getpass
import warnings
warnings.filterwarnings('ignore')

if 'GROQ_API_KEY' not in os.environ:
    os.environ['GROQ_API_KEY'] = getpass("Introduce tu GROQ API Key: ")

print("Configuración completada ✓")

<a name="componentes"></a>
## 3. Componentes de un Agente

### 3.1 Planificación

El agente debe ser capaz de:
- **Descomponer** tareas complejas en subtareas
- **Razonar** sobre qué hacer a continuación
- **Auto-corregirse** si algo falla

### 3.2 Memoria

| Tipo | Descripción | Implementación |
|------|-------------|----------------|
| **Sensorial** | Input actual | El prompt |
| **Corto plazo** | Conversación actual | Chat history |
| **Largo plazo** | Conocimiento persistente | Vector store (RAG) |

### 3.3 Herramientas

Funciones que el agente puede llamar:
- Búsqueda web
- Calculadora
- Ejecución de código
- APIs externas
- Bases de datos

<a name="planificacion"></a>
## 4. Planificación: Chain of Thought y ReAct

### Chain of Thought (CoT)

Pedir al modelo que "piense paso a paso" mejora el razonamiento.

In [ ]:
from langchain_groq import ChatGroq
from langchain import hub

llm = ChatGroq(model_name="llama-3.3-70b-versatile", temperature=0)

# Chain of Thought example
cot_prompt = """Resuelve el siguiente problema paso a paso.

Problema: Una tienda ofrece 20% de descuento en todos los productos.
Si un artículo cuesta 80€, y además hay un cupón de 5€ de descuento adicional,
¿cuánto pagaría el cliente?

Piensa paso a paso:"""

response = llm.invoke(cot_prompt)
print(response.content)

### ReAct: Reasoning + Acting

**ReAct** combina razonamiento con acciones. El patrón es:

```
Thought: [Razonamiento sobre qué hacer]
Action: [Herramienta a usar]
Action Input: [Parámetros para la herramienta]
Observation: [Resultado de la acción]
... (repetir hasta tener respuesta)
Final Answer: [Respuesta al usuario]
```

In [ ]:
# ReAct prompt structure
react_prompt = """Responde la pregunta usando el formato ReAct.

Herramientas disponibles:
- calculator: Para operaciones matemáticas
- search: Para buscar información

Formato:
Thought: [Tu razonamiento]
Action: [Nombre de herramienta]
Action Input: [Input para la herramienta]
Observation: [Resultado - lo simularemos]
... (repetir si es necesario)
Final Answer: [Tu respuesta final]

Pregunta: ¿Cuántos segundos hay en 3 días, 4 horas y 30 minutos?

Comienza:"""

response = llm.invoke(react_prompt)
print(response.content)

<a name="primer"></a>
## 5. Primer Agente Simple

Vamos a crear un agente básico con herramientas personalizadas.

In [ ]:
from langchain.tools import tool

# Define custom tools
@tool
def calculator(expression: str) -> str:
    """Evalúa una expresión matemática. Ejemplo: '2 + 2' o '10 * 5'"""
    try:
        # Safe evaluation
        allowed = set('0123456789+-*/(). ')
        if not all(c in allowed for c in expression):
            return "Error: Expresión inválida"
        result = eval(expression)
        return str(result)
    except Exception as e:
        return f"Error: {e}"

@tool
def get_current_time() -> str:
    """Obtiene la fecha y hora actual."""
    from datetime import datetime
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

@tool
def string_length(text: str) -> str:
    """Calcula la longitud de un texto."""
    return str(len(text))

# List tools
tools = [calculator, get_current_time, string_length]

print("Herramientas disponibles:")
for t in tools:
    print(f"  - {t.name}: {t.description}")

In [ ]:
# Test tools directly
print("Test calculator:", calculator.invoke("15 * 4 + 10"))
print("Test time:", get_current_time.invoke(""))
print("Test length:", string_length.invoke("Hola mundo"))

In [ ]:
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate

# Create agent prompt
agent_prompt = ChatPromptTemplate.from_messages([
    ("system", """Eres un asistente útil con acceso a herramientas.
Usa las herramientas cuando sea necesario para responder preguntas.
Siempre muestra tu razonamiento."""),
    ("placeholder", "{chat_history}"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

# Create the agent
agent = create_tool_calling_agent(llm, tools, agent_prompt)

# Create executor
agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools, 
    verbose=True  # Show reasoning
)

print("Agente creado ✓")

In [ ]:
# Test the agent
result = agent_executor.invoke({
    "input": "¿Cuánto es 25 multiplicado por 4, más 150?",
    "chat_history": []
})

print("\n" + "="*50)
print(f"Respuesta final: {result['output']}")

In [ ]:
# Test with multiple tools
result = agent_executor.invoke({
    "input": "¿Qué hora es y cuántos caracteres tiene la frase 'Inteligencia Artificial'?",
    "chat_history": []
})

print("\n" + "="*50)
print(f"Respuesta final: {result['output']}")

<a name="ejercicios"></a>
## 6. Ejercicios Prácticos

### Ejercicio 1: Crear una herramienta personalizada

In [ ]:
# Exercise 1: Create a custom tool
# Ideas:
# - Temperature converter (Celsius to Fahrenheit)
# - Random number generator
# - Word counter

@tool
def celsius_to_fahrenheit(celsius: str) -> str:
    """Convierte temperatura de Celsius a Fahrenheit. Input: número en Celsius."""
    try:
        c = float(celsius)
        f = (c * 9/5) + 32
        return f"{c}°C = {f}°F"
    except:
        return "Error: Proporciona un número válido"

# Test your tool
print(celsius_to_fahrenheit.invoke("25"))

### Ejercicio 2: Agente con múltiples herramientas

In [ ]:
# Exercise 2: Create an agent with your new tools
# Add the celsius_to_fahrenheit tool and test the agent

# extended_tools = tools + [celsius_to_fahrenheit]
# Create new agent with extended tools
# Test with: "Si la temperatura es 30 grados Celsius, ¿cuánto es en Fahrenheit?"

## Resumen

En este notebook hemos aprendido:

1. **Agentes**: LLMs que pueden razonar, recordar y actuar
2. **Componentes**: Planificación, memoria, herramientas
3. **Chain of Thought**: Razonamiento paso a paso
4. **ReAct**: Combinación de pensamiento y acción
5. **Herramientas**: Funciones que el agente puede llamar

### Arquitectura de un Agente

```
Input → [Planificación] → [Selección de herramienta] → [Ejecución] → [Observación] → [Repetir o Responder]
```

En el siguiente notebook profundizaremos en **Agentes con LangChain**, incluyendo herramientas de búsqueda y RAG.

---

## Referencias

- [LangChain Agents](https://python.langchain.com/docs/modules/agents/)
- [ReAct Paper](https://arxiv.org/abs/2210.03629)
- [Lilian Weng's Agent Blog](https://lilianweng.github.io/posts/2023-06-23-agent/)